In [1]:
import tensorflow as tf
tf.__version__

'2.0.0-rc1'

### 理解卷积

In [60]:
def conv2d(x,kernel):
    h,w = kernel.shape
    out_put = []
    for i in range((x.shape[0] - h +1)):
        for j in range((x.shape[1] - w +1)):
            out_put.append( (tf.reduce_sum(tf.multiply(x[i:i+h,j:j+w] , kernel))))
        
    y= tf.stack(out_put)
        
    return tf.reshape(y,shape=(x.shape[0] - h +1,x.shape[1] - w +1))

In [28]:
x = tf.random.normal(shape=(10,10))

In [29]:
k = tf.random.normal(shape=(3,3))
conv2d(x,k)

<tf.Tensor: id=10057, shape=(8, 8), dtype=float32, numpy=
array([[ 0.7956085 , -1.6489787 , -3.6899586 , -0.44755122, -0.2785045 ,
         3.76672   , -2.5851974 ,  0.26247823],
       [ 4.1905475 , -0.28131947,  0.07861653,  2.9538245 , -2.6310577 ,
        -1.0870335 , -1.1166364 , -2.395783  ],
       [ 3.45217   , -1.4768708 , -0.59862185,  1.9901712 , -0.38292637,
         1.1924441 ,  2.0771327 ,  1.1545573 ],
       [-4.695121  ,  2.3784368 , -1.5360371 , -1.7572236 ,  5.094795  ,
        -0.5646023 ,  0.9559533 , -4.7006397 ],
       [ 2.1050034 ,  2.188803  , -0.8850391 , -0.39543182, -4.1420918 ,
         0.74443966,  0.9988775 ,  2.0866895 ],
       [-3.034708  ,  2.2227004 ,  0.8950124 , -1.9327092 ,  5.025414  ,
        -2.0831928 , -2.6197257 ,  2.0586076 ],
       [ 0.454712  , -1.9518709 , -0.09180117,  2.858404  , -3.5624027 ,
        -0.71505475,  1.4328781 , -1.6332513 ],
       [-0.86923087,  3.4745123 , -2.8528528 ,  1.0565485 , -0.6818204 ,
         2.9156146 , -

### 卷积层

In [33]:
class Conv2D(tf.keras.layers.Layer):
    def __init__(self,kernel,**kwargs):
        super(Conv2D,self).__init__(**kwargs)
        w_init = tf.keras.initializers.he_normal()
        self.k_weight = self.add_weight(shape=(kernel,kernel),initializer=w_init)
        self.b = self.add_weight(shape=(1,),initializer = tf.keras.initializers.ones())
    def call(self,x):
        return conv2d(x,self.k_weight) + self.b    

In [34]:
conv = Conv2D(3)
conv(x)

<tf.Tensor: id=10596, shape=(8, 8), dtype=float32, numpy=
array([[-1.0736337 ,  2.4059558 ,  1.7110702 , -1.9034982 ,  0.30734777,
         0.6368405 ,  3.8713622 ,  0.9441674 ],
       [-2.1219869 , -1.3630974 ,  2.708483  ,  2.7867622 ,  2.4200711 ,
         1.4375907 ,  2.8652477 ,  5.826682  ],
       [ 1.0378562 , -0.14327812,  2.205799  , -0.6308235 ,  0.63816595,
         2.6967561 , -0.55436754,  0.27601868],
       [ 3.116799  , -0.10236311,  4.270503  ,  1.7004194 , -4.0655603 ,
        -1.1938803 , -1.0862429 , -1.6889696 ],
       [ 0.11553031, -1.5358028 ,  0.909774  ,  2.3419552 ,  1.3482902 ,
        -0.886163  , -1.2931616 ,  1.2867411 ],
       [ 0.57553107,  0.58852345,  0.48777962, -0.27321684,  2.7852826 ,
         2.6020215 ,  0.2437458 , -2.4097574 ],
       [ 3.996617  ,  2.5351882 ,  4.4872885 , -4.1353226 , -0.07832599,
         2.168375  ,  1.9111457 ,  0.9150233 ],
       [ 2.791082  ,  0.09927624,  2.6862822 , -3.8503242 ,  0.596218  ,
         2.220161  , -

### 边界检测

In [38]:
import numpy as np

In [39]:
x = np.ones((6,8))
x[:,2:6] = 0

In [40]:
x_in = tf.constant(x)

In [64]:
k = tf.constant([[1,-1]],dtype=tf.float32)

In [49]:
y = conv2d(x,k)

In [50]:
y

<tf.Tensor: id=10953, shape=(6, 7), dtype=int32, numpy=
array([[ 0,  1,  0,  0,  0, -1,  0],
       [ 0,  1,  0,  0,  0, -1,  0],
       [ 0,  1,  0,  0,  0, -1,  0],
       [ 0,  1,  0,  0,  0, -1,  0],
       [ 0,  1,  0,  0,  0, -1,  0],
       [ 0,  1,  0,  0,  0, -1,  0]])>

### 通过数据学习核数组

In [70]:
y = tf.cast(y,tf.float32)
x = tf.cast(x,tf.float32)

In [71]:
X = tf.reshape(x,(6,8))
Y = tf.reshape(y,shape=(6,7))

In [76]:
k2 = tf.random.normal(shape=k.shape)
k2

<tf.Tensor: id=17378, shape=(1, 2), dtype=float32, numpy=array([[ 0.42971647, -0.6225934 ]], dtype=float32)>

In [79]:
for i in range(10):
    with tf.GradientTape() as tape:
        tape.watch(k2)
        y_hat = conv2d(X,k2)
        l = tf.subtract(y_hat , Y)
        l = l**2
    
    w_grad = tape.gradient(l,k2)
    k2 -=w_grad * 3e-2
    
    print('batch %d, loss %.3f' % (i + 1, tf.reduce_sum(l).numpy()))
    

batch 1, loss 3.252
batch 2, loss 1.461
batch 3, loss 0.681
batch 4, loss 0.331
batch 5, loss 0.169
batch 6, loss 0.091
batch 7, loss 0.051
batch 8, loss 0.030
batch 9, loss 0.018
batch 10, loss 0.011


In [80]:
k2

<tf.Tensor: id=23666, shape=(1, 2), dtype=float32, numpy=array([[ 0.98418194, -1.0048919 ]], dtype=float32)>